In [7]:
%matplotlib inline
import torch
import torch.nn as nn
import torchvision
from torch.utils import data
from torchvision import transforms
import matplotlib.pyplot as plt

 # 权重衰减
相当于给损失函数加上$L_2$范数正则化，防止过拟合的产生。

In [8]:
def load_data(batch_size,resize=None):
    '''下载数据集，将其加载到内存中'''
    trans = [transforms.ToTensor()]
    if resize:                       #是否对原始数据进行大小变换处理
        trans.insert(0,transforms.Resize(resize))
    trans = transforms.Compose(trans)
    
    mnist_train = torchvision.datasets.FashionMNIST(
    root="../data", train=True, transform=trans, download=True)    #读取和加在数据集，使用torchvision中内置的数据
    mnist_test = torchvision.datasets.FashionMNIST(
    root="../data", train=False, transform=trans, download=True)
    return (data.DataLoader(mnist_train,batch_size,shuffle=True,num_workers=4),data.DataLoader(mnist_test,batch_size,shuffle=True,num_workers=4))

In [9]:
def get_labels(labels):
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat','sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

In [10]:
#可视化训练样本
def show_image(imgs,num_rows,num_cols, titles=None, scale=1.5):
    #绘制列表
    figsize=(num_cols*scale,num_rows*scale)
    fig,axes = plt.subplots(num_rows,num_cols,figsize=figsize)   #subplot用来创建总画布
    axes=axes.flatten()
    for i,(ax,img) in enumerate(zip(axes,imgs)):
        if torch.is_tensor(img):
            ax.imshow(img.numpy())
        else:
            ax.imshow(img)
#     ax.axes.get_xaxis().set_visible(False)
#     ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes

In [11]:
batch_size=256
train_iter,test_iter=load_data(batch_size)

In [12]:
def evaluate_accuracy(test_iter,net):
    acc_sum,n=0.0,0
    for X,y in test_iter:
        y_hat = net(X)
        acc_sum += (y_hat.argmax(axis=1)==y).sum().item()
        n += y.shape[0]
        
    return acc_sum/n
        

### 初始化参数

In [33]:
def init_params():
    w1 = torch.normal(0,1,(num_inputs,num_hidden), requires_grad = True)
    b1 = torch.zeros(num_hidden,requires_grad=True)
    w2 = torch.normal(0,1,(num_hidden,num_outputs), requires_grad = True)
    b2 = torch.zeros(num_outputs,requires_grad=True)
    return [w1,w2,b1,b2]

In [46]:
def l2_penalty(w):
    return (w**2).mean()/2

In [35]:
num_inputs, num_outputs, num_hidden = 784, 10, 256
batch_size,num_epochs,lr = 1, 100, 0.03
W1,W2,b1,b2 = init_params()

In [36]:
def net(x):
    x = x.reshape(-1,num_inputs)
    H=torch.relu(torch.mm(x,W1)+b1)
    out = torch.mm(H,W2)+b2
    return out

In [37]:
loss = torch.nn.CrossEntropyLoss()
optimizer=torch.optim.SGD([W1,W2,b1,b2],lr=0.1)

In [41]:
num_epochs=10
lr=0.1
def train_ch3(net,train_iter,test_iter,loss,optimizer,num_epochs,batch_size):
    for epoch in range(num_epochs):
        train_l_sum,train_acc_sum,n=0.0,0.0,0
        for X,y in train_iter:
            y_hat=net(X)
            l=loss(y_hat,y).sum()
            
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and paras[0].grad is not None:
                param.grad.data.zero_()
                
            l.backward()
            
            if optimizer is None:
                torch.optim.SGD(params,lr=0.1)
            else:
                optimizer.step()
                
            train_l_sum+= l.item()
            train_acc_sum+= (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            
        test_acc = evaluate_accuracy(test_iter,net)
        print('epoch %d, loss %.4f, train_acc %.3f, test_acc %.3f' %(epoch+1,train_l_sum/n, train_acc_sum/n,test_acc))
        

In [39]:
train_ch3(net,train_iter,test_iter,loss,optimizer,num_epochs,batch_size)

epoch 1, loss 0.0460, train_acc 0.705, test_acc 0.728
epoch 2, loss 0.0084, train_acc 0.748, test_acc 0.717
epoch 3, loss 0.0059, train_acc 0.757, test_acc 0.746
epoch 4, loss 0.0048, train_acc 0.767, test_acc 0.719
epoch 5, loss 0.0042, train_acc 0.774, test_acc 0.746
epoch 6, loss 0.0038, train_acc 0.779, test_acc 0.756
epoch 7, loss 0.0035, train_acc 0.783, test_acc 0.762
epoch 8, loss 0.0032, train_acc 0.788, test_acc 0.764
epoch 9, loss 0.0031, train_acc 0.791, test_acc 0.752
epoch 10, loss 0.0029, train_acc 0.795, test_acc 0.772


### 使用权重衰减

In [47]:
num_epochs=10
lr=0.1
def train_ch3_weight_decay(net,train_iter,test_iter,loss,optimizer,num_epochs,batch_size):
    for epoch in range(num_epochs):
        train_l_sum,train_acc_sum,n=0.0,0.0,0
        for X,y in train_iter:
            y_hat=net(X)
            l=loss(y_hat,y).sum()+ l2_penalty(W1)+l2_penalty(W2)
            
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and paras[0].grad is not None:
                param.grad.data.zero_()
                
            l.backward()
            
            if optimizer is None:
                torch.optim.SGD(params,lr=0.1)
            else:
                optimizer.step()
                
            train_l_sum+= l.item()
            train_acc_sum+= (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            
        test_acc = evaluate_accuracy(test_iter,net)
        print('epoch %d, loss %.4f, train_acc %.3f, test_acc %.3f' %(epoch+1,train_l_sum/n, train_acc_sum/n,test_acc))
         

In [48]:
train_ch3_weight_decay(net,train_iter,test_iter,loss,optimizer,num_epochs,batch_size)

epoch 1, loss 0.0055, train_acc 0.441, test_acc 0.640
epoch 2, loss 0.0035, train_acc 0.674, test_acc 0.675
epoch 3, loss 0.0026, train_acc 0.755, test_acc 0.759
epoch 4, loss 0.0023, train_acc 0.792, test_acc 0.753
epoch 5, loss 0.0021, train_acc 0.812, test_acc 0.791
epoch 6, loss 0.0020, train_acc 0.823, test_acc 0.788
epoch 7, loss 0.0019, train_acc 0.827, test_acc 0.810
epoch 8, loss 0.0019, train_acc 0.832, test_acc 0.818
epoch 9, loss 0.0018, train_acc 0.838, test_acc 0.792
epoch 10, loss 0.0018, train_acc 0.841, test_acc 0.810


使用正则化的方法能够使得loss收敛更快，准确率更高

#### 方法二简单实现：在优化的过程中加入weight_decay, 进行权重的衰减

In [60]:
optimizer=torch.optim.SGD([{'params':[W1,W2], 'weight_decay':0.01},
                           {'params':[b1,b2]}],lr=0.1)

In [61]:
train_ch3(net,train_iter,test_iter,loss,optimizer,num_epochs,batch_size)

epoch 1, loss 0.0017, train_acc 0.845, test_acc 0.830
epoch 2, loss 0.0018, train_acc 0.843, test_acc 0.826
epoch 3, loss 0.0018, train_acc 0.840, test_acc 0.818
epoch 4, loss 0.0019, train_acc 0.838, test_acc 0.813
epoch 5, loss 0.0019, train_acc 0.836, test_acc 0.811
epoch 6, loss 0.0019, train_acc 0.834, test_acc 0.817
epoch 7, loss 0.0019, train_acc 0.835, test_acc 0.798
epoch 8, loss 0.0019, train_acc 0.834, test_acc 0.821
epoch 9, loss 0.0019, train_acc 0.836, test_acc 0.823
epoch 10, loss 0.0019, train_acc 0.834, test_acc 0.822


到目前为止，我们只接触到一个简单线性函数的概念。
此外，由什么构成一个简单的非线性函数可能是一个更复杂的问题。
例如，[再生核希尔伯特空间（RKHS）](https://en.wikipedia.org/wiki/Reproducing_kernel_Hilbert_space)
允许在非线性环境中应用为线性函数引入的工具。
不幸的是，基于RKHS的算法往往难以应用到大型、高维的数据。
在这本书中，我们将默认使用简单的启发式方法，即在深层网络的所有层上应用权重衰减。

## 小结

* 正则化是处理过拟合的常用方法：在训练集的损失函数中加入惩罚项，以降低学习到的模型的复杂度。
* 保持模型简单的一个特别的选择是使用$L_2$惩罚的权重衰减。这会导致学习算法更新步骤中的权重衰减。
* 权重衰减功能在深度学习框架的优化器中提供。
* 在同一训练代码实现中，不同的参数集可以有不同的更新行为。

## 练习

1. 在本节的估计问题中使用$\lambda$的值进行实验。绘制训练和测试精度关于$\lambda$的函数。观察到了什么？
1. 使用验证集来找到最佳值$\lambda$。它真的是最优值吗？这有关系吗？
1. 如果我们使用$\sum_i |w_i|$作为我们选择的惩罚（$L_1$正则化），那么更新方程会是什么样子？
1. 我们知道$\|\mathbf{w}\|^2 = \mathbf{w}^\top \mathbf{w}$。能找到类似的矩阵方程吗（见 :numref:`subsec_lin-algebra-norms` 中的Frobenius范数）？
1. 回顾训练误差和泛化误差之间的关系。除了权重衰减、增加训练数据、使用适当复杂度的模型之外，还能想出其他什么方法来处理过拟合？
1. 在贝叶斯统计中，我们使用先验和似然的乘积，通过公式$P(w \mid x) \propto P(x \mid w) P(w)$得到后验。如何得到带正则化的$P(w)$？


# 丢弃法dropout
训练过程中丢弃掉一些神经元


### 从0开始实现

In [62]:
def dropout_layer(X, dropout):
    assert 0 <= dropout <= 1
    if dropout==1:     #全部丢弃
        return torch.zeros_like(X)
    if dropout==0:     #全部保留
        return X
    mask = (torch.rand(X.shape)>dropout).float()
    return mask*X/(1-dropout)

In [64]:
X= torch.arange(16, dtype = torch.float32).reshape((2, 8))
X

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])

In [77]:
print(dropout_layer(X,0))

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])


In [78]:
print(dropout_layer(X,0.5))  

tensor([[ 0.,  2.,  0.,  6.,  0.,  0., 12., 14.],
        [16.,  0., 20.,  0., 24., 26., 28., 30.]])


In [79]:
print(dropout_layer(X,1))

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


#### 定义模型参数

In [80]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256

In [81]:
dropout1, dropout2 = 0.2,0.5

In [87]:
class Net(nn.Module):
    def __init__(self, num_inputs,num_outputs,num_hiddens1,num_hiddens2,is_training=True):
        super(Net,self).__init__()
        self.num_inputs=num_inputs
        self.is_training=is_training
        self.lin1 = nn.Linear(num_inputs,num_hiddens1)
        self.lin2 = nn.Linear(num_hiddens1,num_hiddens2)
        self.lin3 = nn.Linear(num_hiddens2,num_outputs)
        self.relu=nn.ReLU()
    
    def forward(self,X):
        H1 = self.relu(self.lin1(X.reshape((-1,self.num_inputs))))
        
        if self.is_training == True:
            H1 = dropout_layer(H1,dropout1)
        H2 = self.relu(self.lin2(H1))
        
        if self.is_training == True:
            H2 = dropout_layer(H2,dropout2)
        out = self.lin3(H2)
        return out
    

In [88]:
net = Net(num_inputs, num_outputs, num_hiddens1, num_hiddens2)

#### 训练和测试

In [89]:
num_epochs, lr, batch_size = 10, 0.5, 256
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=lr)

In [90]:
train_ch3(net,train_iter,test_iter,loss,optimizer,num_epochs,batch_size)

epoch 1, loss 0.0034, train_acc 0.670, test_acc 0.780
epoch 2, loss 0.0021, train_acc 0.804, test_acc 0.804
epoch 3, loss 0.0018, train_acc 0.833, test_acc 0.829
epoch 4, loss 0.0017, train_acc 0.843, test_acc 0.839
epoch 5, loss 0.0016, train_acc 0.853, test_acc 0.848
epoch 6, loss 0.0015, train_acc 0.862, test_acc 0.839
epoch 7, loss 0.0014, train_acc 0.865, test_acc 0.841
epoch 8, loss 0.0014, train_acc 0.871, test_acc 0.860
epoch 9, loss 0.0013, train_acc 0.875, test_acc 0.859
epoch 10, loss 0.0013, train_acc 0.877, test_acc 0.841


### 简洁实现

In [97]:
net = nn.Sequential(nn.Flatten(),
                   nn.Linear(num_inputs,num_hiddens1),
                   nn.ReLU(),
                   nn.Dropout(dropout1),
                   nn.Linear(num_hiddens1,num_hiddens2),
                   nn.ReLU(),
                   nn.Dropout(dropout2),
                   nn.Linear(num_hiddens2,num_outputs))

In [99]:
def init_weights(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,std=0.01)
net.apply(init_weights)    

Sequential(
  (0): Flatten()
  (1): Linear(in_features=784, out_features=256, bias=True)
  (2): ReLU()
  (3): Dropout(p=0.2, inplace=False)
  (4): Linear(in_features=256, out_features=256, bias=True)
  (5): ReLU()
  (6): Dropout(p=0.5, inplace=False)
  (7): Linear(in_features=256, out_features=10, bias=True)
)

In [100]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=lr)

In [101]:
train_ch3(net,train_iter,test_iter,loss,optimizer,num_epochs,batch_size)

epoch 1, loss 0.0047, train_acc 0.536, test_acc 0.737
epoch 2, loss 0.0023, train_acc 0.787, test_acc 0.780
epoch 3, loss 0.0019, train_acc 0.822, test_acc 0.814
epoch 4, loss 0.0017, train_acc 0.839, test_acc 0.826
epoch 5, loss 0.0016, train_acc 0.848, test_acc 0.838
epoch 6, loss 0.0016, train_acc 0.853, test_acc 0.825
epoch 7, loss 0.0015, train_acc 0.861, test_acc 0.822
epoch 8, loss 0.0014, train_acc 0.864, test_acc 0.843
epoch 9, loss 0.0014, train_acc 0.869, test_acc 0.833
epoch 10, loss 0.0014, train_acc 0.872, test_acc 0.849


## 小结

* 暂退法在前向传播过程中，计算每一内部层的同时丢弃一些神经元。
* 暂退法可以避免过拟合，它通常与控制权重向量的维数和大小结合使用的。
* 暂退法将活性值$h$替换为具有期望值$h$的随机变量。
* 暂退法仅在训练期间使用。

## 练习

1. 如果更改第一层和第二层的暂退法概率，会发生什么情况？具体地说，如果交换这两个层，会发生什么情况？设计一个实验来回答这些问题，定量描述该结果，并总结定性的结论。
1. 增加训练轮数，并将使用暂退法和不使用暂退法时获得的结果进行比较。
1. 当应用或不应用暂退法时，每个隐藏层中激活值的方差是多少？绘制一个曲线图，以显示这两个模型的每个隐藏层中激活值的方差是如何随时间变化的。
1. 为什么在测试时通常不使用暂退法？
1. 以本节中的模型为例，比较使用暂退法和权重衰减的效果。如果同时使用暂退法和权重衰减，会发生什么情况？结果是累加的吗？收益是否减少（或者说更糟）？它们互相抵消了吗？
1. 如果我们将暂退法应用到权重矩阵的各个权重，而不是激活值，会发生什么？
1. 发明另一种用于在每一层注入随机噪声的技术，该技术不同于标准的暂退法技术。尝试开发一种在Fashion-MNIST数据集（对于固定架构）上性能优于暂退法的方法。
